## The Needleman-Wunsch algorithm
We will try to reproduce the FAST/FAT alignment in the slides

In [1]:
seq_i = 'FAST' # verically (left)
seq_j = 'FAT'  # horizontally (top)

This will be our scoring system:

In [2]:
match = 2
mismatch = -1
gep = -1

Initialize the matrix of scores with all 0s. Use a list of lists. It will be easier (if I have to help you) if we all put *seq_i* vertically and *seq_j* horizontally. You should get something like this (**without the letters**).

In [3]:
score_mat = [[0] * (len(seq_j) + 1) for _ in range(len(seq_i) + 1)]

Check the content of *score_mat* by running the following cell, which will add the letters in an easy way.

In [5]:
import pandas
labels_i = list('-' + seq_i)
labels_j = list('-' + seq_j)
pandas.DataFrame(score_mat, labels_i, labels_j)

,-,F,A,T
-,0,0,0,0
F,0,0,0,0
A,0,0,0,0
S,0,0,0,0
T,0,0,0,0


Initialize the traceback matrix with all 0s. Agin, use a list of lists.

In [7]:
traceback = [[0] * (len(seq_j) + 1) for x in range(len(seq_i) + 1)]

As before, check that you have what you expect in the *traceback* matrix

In [8]:
pandas.DataFrame(traceback, labels_i, labels_j)

,-,F,A,T
-,0,0,0,0
F,0,0,0,0
A,0,0,0,0
S,0,0,0,0
T,0,0,0,0


Finish the initialization of *score_mat* and *traceback* (first row and first column)

In [9]:
for i in range(1, len(seq_i) + 1):
    score_mat[i][0] = score_mat[i-1][0] + gep
    traceback[i][0] = 1 #arrow pointing up

for j in range(1, len(seq_j) + 1):
    score_mat[0][j] = score_mat[0][j-1] + gep
    traceback[0][j] = -1 #arrow pointing left

Check the current state of *score_mat* and *traceback*

In [10]:
pandas.DataFrame(score_mat, labels_i, labels_j)

,-,F,A,T
-,0,-1,-2,-3
F,-1,0,0,0
A,-2,0,0,0
S,-3,0,0,0
T,-4,0,0,0


In [11]:
pandas.DataFrame(traceback, labels_i, labels_j)

,-,F,A,T
-,0,-1,-1,-1
F,1,0,0,0
A,1,0,0,0
S,1,0,0,0
T,1,0,0,0


We are ready to fill both *score_mat* and *traceback* matrices. But first you will have to fill the gaps in the code:

In [12]:
for i in range(1, len(seq_i) + 1):
    for j in range(1, len(seq_j) + 1):
        if seq_i[i - 1] == seq_j[j - 1]:
            score = match
        else:
            score = mismatch
        
        subst = score_mat[i - 1][j - 1] + score
        inser = score_mat[i][j - 1] + gep
        delet = score_mat[i - 1][j] + gep

        if subst >= inser and subst >= delet:
            score_mat[i][j] = subst
            traceback[i][j] = 0
        elif inser >= delet:
            score_mat[i][j] = inser
            traceback[i][j] = -1
        else:
            score_mat[i][j] = delet
            traceback[i][j] = 1

Examine the code above as it is the core of the dynamic program algorithm  

Check the content of *score_mat* now:

In [13]:
pandas.DataFrame(score_mat, labels_i, labels_j)

,-,F,A,T
-,0,-1,-2,-3
F,-1,2,1,0
A,-2,1,4,3
S,-3,0,3,3
T,-4,-1,2,5


And the content of *traceback*:

In [14]:
pandas.DataFrame(traceback, labels_i, labels_j)

,-,F,A,T
-,0,-1,-1,-1
F,1,0,-1,-1
A,1,1,0,-1
S,1,1,1,0
T,1,1,1,0


**Q:** Where is the optimal score of the alignment between the two sequences?

In [ ]:
#The optimal score is in the bottom right cell

Now let's get the alignment. We will have to do the traceback and fill the gaps!

In [15]:
# We initialize the aligned sequences
aln_i = []
aln_j = []

# We start the traceback at the bottom-right corner of the table
i = len(seq_i)
j = len(seq_j)

# We end the traceback at the upper-right corner of the table
while i != 0 or j != 0:
    if traceback[i][j] == 0:
        aln_i.append(seq_i[i - 1])
        aln_j.append(seq_j[j - 1])
        i -= 1
        j -= 1
    elif traceback[i][j] == -1:
        aln_i.append('-')
        aln_j.append(seq_j[j - 1])
        j -= 1
    else:
        aln_i.append(seq_i[i - 1])
        aln_j.append('-')
        i -= 1
alignment_i = ''.join(aln_i[::-1])
alignment_j = ''.join(aln_j[::-1])

print(alignment_i)
print(alignment_j)

FAST
FA-T


The alignment is done now. Or not???!!!!  
Print the actual alignment as two strings.

In [16]:
print(alignment_i)
print(alignment_i)

FAST
FAST


With the provided *match*, *mismatch* and *gep* values you should get the following:

## Exercise: nw1

Create a function to run the NW algorithm more conviniently.

In [17]:
import pandas as pd

def nw(seq_i, seq_j, match, mismatch, gap):
    # Start matrices
    score_mat = [[0] * (len(seq_j) + 1) for _ in range(len(seq_i) + 1)]
    traceback = [[0] * (len(seq_j) + 1) for _ in range(len(seq_i) + 1)]
    
    for i in range(1, len(seq_i) + 1):
        score_mat[i][0] = gap * i
        traceback[i][0] = 1  # arrow pointing up

    for j in range(1, len(seq_j) + 1):
        score_mat[0][j] = gap * j
        traceback[0][j] = -1  # arrow pointing left

    # Fill matrices
    for i in range(1, len(seq_i) + 1):
        for j in range(1, len(seq_j) + 1):
            if seq_i[i - 1] == seq_j[j - 1]:
                score = match
            else:
                score = mismatch
            
            subst = score_mat[i - 1][j - 1] + score
            inser = score_mat[i][j - 1] + gap
            delet = score_mat[i - 1][j] + gap

            if subst > inser and subst > delet:
                score_mat[i][j] = subst
                traceback[i][j] = 0
            elif inser > delet:
                score_mat[i][j] = inser
                traceback[i][j] = -1
            else:
                score_mat[i][j] = delet
                traceback[i][j] = 1
    
    # Perform traceback to get alignment
    aln_i = []
    aln_j = []
    i, j = len(seq_i), len(seq_j)
    while i != 0 or j != 0:
        if traceback[i][j] == 0:
            aln_i.append(seq_i[i - 1])
            aln_j.append(seq_j[j - 1])
            i -= 1
            j -= 1
        elif traceback[i][j] == -1:
            aln_i.append('-')
            aln_j.append(seq_j[j - 1])
            j -= 1
        else:
            aln_i.append(seq_i[i - 1])
            aln_j.append('-')
            i -= 1
    
    # Reverse alignments 
    aln_i.reverse()
    aln_j.reverse()
    
    return ''.join(aln_i), ''.join(aln_j)

seq1 = 'THEFASTCAT'
seq2 = 'THERAT'
match = 1
mismatch = -8
gap = -4
alignment_i, alignment_j = nw(seq1, seq2, match, mismatch, gap)
print("Alignment:")
print(alignment_i)
print(alignment_j)


Alignment:
THE-FASTCAT
THER-A-T---


 Tests are in the exercise collection. Continue one you have a function that passes all tests.

We will use the fowlloing sequences and parameters

In [ ]:
seq1 = 'THEFASTCAT'
seq2 = 'THERAT'

match = 1
mismatch = -8
gep = -4

Run the **nw()** function and display the alignment

**Q:** Provinding the 'THEFASTCAT' as seq1 and 'THERAT' as seq2 does not provide the same alignment as if you do it in the opposite order. Why?

**Q:** Why do think we do not obtain the alignment shown below, which is one column shorter? 

Get the alignment above by changing *mismatch* and *gep*